
This is for data processing/calculations: 
1. The fastest way for data calculation is to use vectorization.(using loc, combined with boolean mask which is filtered at the index level, and then select the column.)
2. Second is to use apply
3. Lastly, using looping, iterrows. 
4. when you add magic command, there will be no other output other than what you commanded.

This is for data cleaning, decrease the size of the data file: 





In [1]:
import pandas as pd
import numpy as np

# Create our Dataset

In [2]:
def get_data(size = 10000):
    df = pd.DataFrame()
    df['age'] = np.random.randint(0, 100, size)
    df["time_in_bed"] = np.random.randint(0, 9, size)
    df["pct_sleeping"] = np.random.rand(size)
    df["favorite_food"] = np.random.choice(['pizza', 'taco', 'ice-cream'], size)
    df["hate_food"] = np.random.choice(['broccoli','candy corn','eggs'], size)
    return df

reward calculation: 
- If they were in bed for more than 5 hours and they were sleeping for more than 50%, we give them their favorite food. 
- otherwise, we give them their hate good. 
- if they are over 90 years old give their favorite food regardless.


In [3]:
def reward_cal(row):
    if row['age']>=90:
        return row['favorite_food']
    if (row["time_in_bed"] >5 )& (row["pct_sleeping"] > 0.5):
        return row["favorite_food"]
    return row["hate_food"]

# Level 1 - Using Iterations


In [12]:
# %%timeit # note if you add the magic command here, it won't output anything but the running time. 

df = get_data()
for index, row in df.iterrows():
    df.loc[index, 'reward'] = reward_cal(row)
df

1.06 s ± 27.3 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [5]:
print(df)

NameError: name 'df' is not defined

# level 2 - Using pd.apply() method


In [13]:
df1 = get_data()
df1['reward'] = df.apply(reward_cal, axis = 1)

In [14]:
df1

,age,time_in_bed,pct_sleeping,favorite_food,hate_food,reward
0,39,3,0.469246,pizza,broccoli,candy corn
1,91,2,0.570891,ice-cream,eggs,candy corn
2,40,5,0.487683,taco,candy corn,eggs
3,28,7,0.725172,taco,broccoli,candy corn
4,88,7,0.858404,ice-cream,eggs,broccoli
...,...,...,...,...,...,...
9995,21,1,0.012305,taco,broccoli,eggs
9996,95,1,0.192751,ice-cream,broccoli,eggs
9997,77,1,0.604307,taco,candy corn,candy corn
9998,5,5,0.782849,pizza,broccoli,ice-cream


# Level 3 - Vectorized fuctions

In [15]:
((df['pct_sleeping'] > 0.5) & (df['time_in_bed'] > 5)) | (df['age'] > 90)

0       False
1       False
2       False
3       False
4       False
        ...  
9995    False
9996    False
9997    False
9998     True
9999    False
Length: 10000, dtype: bool

In [20]:
%%timeit
df = get_data()
df['reward'] = df['hate_food']
df.loc[((df['pct_sleeping'] > 0.5) & 
        (df['time_in_bed'] > 5)) | 
        (df['age'] > 90), 'reward'] = df['favorite_food']

6.51 ms ± 253 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


# Efficient Memory use in Pandas


In [35]:
def get_data(size):
    df = pd.DataFrame()
    df['position'] = np.random.choice(['left','middle','right'],size)
    df['age'] = np.random.randint(1, 50, size)
    df['team'] = np.random.choice(['red','blue','yellow','green'], size)
    df['win'] = np.random.choice(['yes','no'], size)
    df['prob'] = np.random.uniform(0, 1, size)
    return df

In [36]:
df = get_data(1000000)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000000 entries, 0 to 999999
Data columns (total 5 columns):
 #   Column    Non-Null Count    Dtype  
---  ------    --------------    -----  
 0   position  1000000 non-null  object 
 1   age       1000000 non-null  int32  
 2   team      1000000 non-null  object 
 3   win       1000000 non-null  object 
 4   prob      1000000 non-null  float64
dtypes: float64(1), int32(1), object(3)
memory usage: 34.3+ MB


In [26]:
%timeit df['age_rank'] = df.groupby(['team','position'])['age'].rank()
%timeit df['prob_rank'] = df.groupby(['team','position'])['prob'].rank()
%timeit df['win_prob_rank'] = df.groupby(['team','position', 'win'])['prob'].rank()

778 ms ± 20.4 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
908 ms ± 121 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
1.51 s ± 184 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [29]:
df[0:100] # need to make sure you understand the group by functio and what's the output look like. 

,position,age,team,win,prob,age_rank,prob_rank,win_prob_rank
0,middle,45,red,no,0.331099,75951.0,27743.0,13781.0
1,middle,20,red,yes,0.923938,33229.0,77373.0,38862.0
2,middle,1,blue,yes,0.581535,826.0,48427.0,24312.0
3,right,25,blue,no,0.221528,41829.5,18557.0,9216.0
4,left,8,yellow,no,0.028998,12665.5,2396.0,1205.0
...,...,...,...,...,...,...,...,...
95,right,45,red,yes,0.512822,75556.0,42776.0,21382.0
96,left,15,blue,no,0.229952,24358.0,19113.0,9564.0
97,middle,17,green,yes,0.980846,28140.0,82171.0,41165.0
98,right,20,red,no,0.584908,32989.0,48620.0,24330.0


In [33]:
type(df.loc[0,'position'])

str

In [38]:
df['position'] = df['position'].astype('category')
df['team'] = df['team'].astype('category')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000000 entries, 0 to 999999
Data columns (total 5 columns):
 #   Column    Non-Null Count    Dtype   
---  ------    --------------    -----   
 0   position  1000000 non-null  category
 1   age       1000000 non-null  int32   
 2   team      1000000 non-null  category
 3   win       1000000 non-null  object  
 4   prob      1000000 non-null  float64 
dtypes: category(2), float64(1), int32(1), object(1)
memory usage: 21.0+ MB


# int downcasting value range

In [ ]:
# int8 can store integers form -128 to 127
# int16 can store integers from -32k to 32k

In [40]:
print(df['age'].min())
print(df['age'].max())

1
49


In [42]:
df['age'] = df['age'].astype('int8')
df['prob'] = df['prob'].astype('float32')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000000 entries, 0 to 999999
Data columns (total 5 columns):
 #   Column    Non-Null Count    Dtype   
---  ------    --------------    -----   
 0   position  1000000 non-null  category
 1   age       1000000 non-null  int8    
 2   team      1000000 non-null  category
 3   win       1000000 non-null  object  
 4   prob      1000000 non-null  float32 
dtypes: category(2), float32(1), int8(1), object(1)
memory usage: 14.3+ MB


# Casting bool (true/false)

In [43]:
df['win'] = df['win'].astype(bool)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000000 entries, 0 to 999999
Data columns (total 5 columns):
 #   Column    Non-Null Count    Dtype   
---  ------    --------------    -----   
 0   position  1000000 non-null  category
 1   age       1000000 non-null  int8    
 2   team      1000000 non-null  category
 3   win       1000000 non-null  bool    
 4   prob      1000000 non-null  float32 
dtypes: bool(1), category(2), float32(1), int8(1)
memory usage: 7.6 MB


# Summary set_dtypes() function

In [50]:
def set_dtypes(df):
    df['position'] = df['position'].astype('category')
    df['team'] = df['team'].astype('category')
    df['age'] = df['age'].astype('int8')
    df['prob'] = df['prob'].astype('float32')
    df['win'] = df['win'].astype("bool")
    return df

In [46]:
df = get_data(10_000_000)
%timeit df['age_rank'] = df.groupby(['team','position'])['age'].rank()
%timeit df['prob_rank'] = df.groupby(['team','position'])['prob'].rank()
%timeit df['win_prob_rank'] = df.groupby(['team','position', 'win'])['prob'].rank()

15.3 s ± 1.34 s per loop (mean ± std. dev. of 7 runs, 1 loop each)
17.4 s ± 1.6 s per loop (mean ± std. dev. of 7 runs, 1 loop each)
16.6 s ± 222 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [45]:
df

,position,age,team,win,prob,age_rank,prob_rank,win_prob_rank
0,right,19,green,True,0.366544,31393.0,30248.0,30248.0
1,right,11,red,True,0.019113,17526.5,1543.0,1543.0
2,left,28,blue,True,0.613237,46730.5,51124.0,51124.0
3,right,23,blue,True,0.375346,38065.0,31163.0,31163.0
4,left,3,green,True,0.431554,4252.0,35805.0,35805.0
...,...,...,...,...,...,...,...,...
999995,left,14,green,True,0.911444,22985.5,75985.0,75985.0
999996,middle,19,green,True,0.658459,31029.5,54831.0,54831.0
999997,right,41,green,True,0.325279,68770.5,26850.0,26850.0
999998,middle,29,red,True,0.525861,48255.0,43447.0,43447.0


In [51]:
df = get_data(10_000_000)
df = set_dtypes(df)
%timeit df['age_rank'] = df.groupby(['team','position'])['age'].rank()
%timeit df['prob_rank'] = df.groupby(['team','position'])['prob'].rank()
%timeit df['win_prob_rank'] = df.groupby(['team','position', 'win'])['prob'].rank()

<magic-timeit>:1: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
<magic-timeit>:1: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.


9.84 s ± 156 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


<magic-timeit>:1: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.


13.8 s ± 139 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


<magic-timeit>:1: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.


14 s ± 259 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


# Other methods

In [ ]:
1. use nrows when reading data for a sample to understand the data. # use df.info(verbose = False, memory_usage = 'deep') to check the summary info.  
2. select only relevant columns in the dataset. 
3. check the max and min of the dataset, use df.describe(), then convert certain datatypes.
4. You can wrap up the above and summarize them when read_csv: df = pd.read_csv("xx", usecols = req_cols, dtype = {'Region':'category', 'Units':'int16'})


# Loading data with chunks

In [ ]:
chunks = pd.read_csv('', iterator = True, chunksize - 1000)

# CSV

In [1]:
# if the user is an excel user, save the file as a csv. file is better. 

NameError: name 'df' is not defined

# Pickle file

In [ ]:
pd.to_pickle()
pd.read_pickle()

# Parquet file

In [ ]:
pip install pyarrow
pip install fastparquet
pd.to_parquet()
pd.read_parquet()

# Feather file

In [2]:
# better of short-term storage.